In [2]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA


In [4]:
data = pd.read_csv("listing.csv", index_col = "listing_id",  encoding='utf-8')
# df = df.drop(["Name", "Ticket", "Cabin"], axis = 1)
# data
# data = data.drop(["agent_address", "agent_logo", "agent_name", "agent_phone", "country_code",
#                   "description", "details_url", "thumbnail_url", "country", "category",
#                   "floor_area", "floor_plan", "furnished_state", "image_150_113_url", "image_354_255_url",
#                   "image_50_38_url", "image_645_430_url", "image_80_60_url", "image_caption", "image_url",
#                   "listing_status", "image_url", "letting_fees", "location_is_approximate", "price_modifier", 
#                   "status", "short_description", "price_change", "price_change_summary", "post_town",
#                   "displayable_address", "first_published_date", "last_published_date", "street_name"], axis = 1)

data = data[{"county", "latitude", "longitude", "new_home", "num_bathrooms",
            "num_bedrooms", "num_floors", "num_recepts", "outcode", "property_type", "price"}]
data




,new_home,price,outcode,county,num_floors,property_type,num_bathrooms,num_recepts,longitude,num_bedrooms,latitude
listing_id,,,,,,,,,,,
48797779,NaN,3000000,BS20,Bristol,0,Detached house,0,5,-2.799919,5,51.465576
50785419,NaN,2975000,TA24,Somerset,0,Farm,0,0,-3.566586,0,51.115680
43906200,NaN,2250000,TA18,Somerset,3,Detached house,3,4,-2.749436,6,50.882690
26664456,NaN,1875000,BA22,Somerset,0,Equestrian property,5,4,-2.638569,8,50.916737
48905557,NaN,1850000,TA21,Somerset,0,Detached house,5,8,-3.237185,5,51.000244
51101347,NaN,0,TA24,Somerset,0,Land,10,10,-3.491858,20,51.200190
48580285,NaN,1750000,BA6,Somerset,0,Detached house,2,3,-2.612244,6,51.109863
48627348,NaN,1750000,TA21,Somerset,0,Detached house,3,4,-3.322501,6,50.958366
50786439,NaN,1700000,TA24,Somerset,0,Farm,0,0,-3.566586,0,51.115680


In [25]:
# Converting string values into numbers
def convert_str(dataset, col_name, value, replacement):
    dataset.loc[dataset[col_name] == value, col_name] = replacement

# Filling none values with 0s
def fill_none(dataset, col_name, replacement):
    dataset[col_name].fillna(replacement, inplace = True)

# Clean up data by filling none, converting string and get rid of 0 prices
fill_none(data, "new_home", 0)
convert_str(data, "new_home", True, 1)
data.drop(data[data.price == 0].index, inplace=True)
data.drop(data[data.property_type.isna()].index, inplace=True)

# Listing all unique counties and it's postcode
county_code = {}
outcode_list = []
county_notna = data[data["county"].notna()]

for i in county_notna["county"].unique():
    for j in county_notna[county_notna["county"] == i]["outcode"].unique():
        outcode_list.append(j)
    county_code[i] = outcode_list
    outcode_list = []

# Get all county's N/A data
county_na = data[data["county"].isna()]

# Check and assign county to data using postcode
county_na_index = county_na.index
for i in county_na_index:
    for key, value in county_code.items():
        if data.loc[[i], "outcode"].values in value:
            data.loc[[i], "county"] = key


In [11]:
# Reengineer property_type into friendly read format(int) for machine to train
dummy_property = pd.get_dummies(data["property_type"])
dummy_county = pd.get_dummies(data["county"])
dummied_data = pd.concat([data, dummy_property, dummy_county], axis = 1)

dummied_data


,num_recepts,county,latitude,num_floors,new_home,price,num_bedrooms,property_type,outcode,longitude,...,Terraced house,Town house,Villa,Bath & N E Somerset,Bristol,Devon,Dorset,North Somerset,Somerset,Wiltshire
listing_id,,,,,,,,,,,,,,,,,,,,,
48797779,5,Bristol,51.465576,0,0,3000000,5,Detached house,BS20,-2.799919,...,0,0,0,0,1,0,0,0,0,0
50785419,0,Somerset,51.115680,0,0,2975000,0,Farm,TA24,-3.566586,...,0,0,0,0,0,0,0,0,1,0
43906200,4,Somerset,50.882690,3,0,2250000,6,Detached house,TA18,-2.749436,...,0,0,0,0,0,0,0,0,1,0
26664456,4,Somerset,50.916737,0,0,1875000,8,Equestrian property,BA22,-2.638569,...,0,0,0,0,0,0,0,0,1,0
48905557,8,Somerset,51.000244,0,0,1850000,5,Detached house,TA21,-3.237185,...,0,0,0,0,0,0,0,0,1,0
48580285,3,Somerset,51.109863,0,0,1750000,6,Detached house,BA6,-2.612244,...,0,0,0,0,0,0,0,0,1,0
48627348,4,Somerset,50.958366,0,0,1750000,6,Detached house,TA21,-3.322501,...,0,0,0,0,0,0,0,0,1,0
50786439,0,Somerset,51.115680,0,0,1700000,0,Farm,TA24,-3.566586,...,0,0,0,0,0,0,0,0,1,0
51722280,3,Somerset,51.148520,0,0,1700000,6,Detached house,BA6,-2.710659,...,0,0,0,0,0,0,0,0,1,0


In [12]:
# Finished using outcode data to assign counties into properties
# and finished reengineering property_type and county
dummied_data = dummied_data.drop(["outcode", "property_type", "county"], axis = 1)

dummied_data

,num_recepts,latitude,num_floors,new_home,price,num_bedrooms,longitude,num_bathrooms,Barn conversion,Block of flats,...,Terraced house,Town house,Villa,Bath & N E Somerset,Bristol,Devon,Dorset,North Somerset,Somerset,Wiltshire
listing_id,,,,,,,,,,,,,,,,,,,,,
48797779,5,51.465576,0,0,3000000,5,-2.799919,0,0,0,...,0,0,0,0,1,0,0,0,0,0
50785419,0,51.115680,0,0,2975000,0,-3.566586,0,0,0,...,0,0,0,0,0,0,0,0,1,0
43906200,4,50.882690,3,0,2250000,6,-2.749436,3,0,0,...,0,0,0,0,0,0,0,0,1,0
26664456,4,50.916737,0,0,1875000,8,-2.638569,5,0,0,...,0,0,0,0,0,0,0,0,1,0
48905557,8,51.000244,0,0,1850000,5,-3.237185,5,0,0,...,0,0,0,0,0,0,0,0,1,0
48580285,3,51.109863,0,0,1750000,6,-2.612244,2,0,0,...,0,0,0,0,0,0,0,0,1,0
48627348,4,50.958366,0,0,1750000,6,-3.322501,3,0,0,...,0,0,0,0,0,0,0,0,1,0
50786439,0,51.115680,0,0,1700000,0,-3.566586,0,0,0,...,0,0,0,0,0,0,0,0,1,0
51722280,3,51.148520,0,0,1700000,6,-2.710659,4,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
dummied_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5312 entries, 48797779 to 50202060
Data columns (total 44 columns):
num_recepts               5312 non-null int64
latitude                  5312 non-null float64
num_floors                5312 non-null int64
new_home                  5312 non-null int64
price                     5312 non-null int64
num_bedrooms              5312 non-null int64
longitude                 5312 non-null float64
num_bathrooms             5312 non-null int64
Barn conversion           5312 non-null uint8
Block of flats            5312 non-null uint8
Bungalow                  5312 non-null uint8
Chalet                    5312 non-null uint8
Cottage                   5312 non-null uint8
Country house             5312 non-null uint8
Detached bungalow         5312 non-null uint8
Detached house            5312 non-null uint8
End terrace house         5312 non-null uint8
Equestrian property       5312 non-null uint8
Farm                      5312 non-null uint8
Farm

In [14]:
# Normalisation
def normalisation(dataset, exception):
    dataNorm = ((dataset - dataset.min()) / (dataset.max() - dataset.min()))
    if exception in dataset:
        dataNorm[exception] = dataset[exception]
    return dataNorm

normalised_data = normalisation(dummied_data, "price")
normalised_data



,num_recepts,latitude,num_floors,new_home,price,num_bedrooms,longitude,num_bathrooms,Barn conversion,Block of flats,...,Terraced house,Town house,Villa,Bath & N E Somerset,Bristol,Devon,Dorset,North Somerset,Somerset,Wiltshire
listing_id,,,,,,,,,,,,,,,,,,,,,
48797779,0.454545,0.983725,0.00,0.0,3000000,0.3125,0.634835,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
50785419,0.000000,0.451363,0.00,0.0,2975000,0.0000,0.125135,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
43906200,0.363636,0.096873,0.75,0.0,2250000,0.3750,0.668397,0.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
26664456,0.363636,0.148675,0.00,0.0,1875000,0.5000,0.742105,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48905557,0.727273,0.275729,0.00,0.0,1850000,0.3125,0.344129,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48580285,0.272727,0.442513,0.00,0.0,1750000,0.3750,0.759606,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48627348,0.363636,0.212013,0.00,0.0,1750000,0.3750,0.287409,0.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
50786439,0.000000,0.451363,0.00,0.0,1700000,0.0000,0.125135,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
51722280,0.272727,0.501329,0.00,0.0,1700000,0.3750,0.694177,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Saving cleaned data and normalised data into a file
dummied_data.to_csv(path_or_buf = "/Users/hoanjinan_otoko/Desktop/epam-ds-training/greenfox/hoanjinan/week-07/cleaned_data.csv")
normalised_data.to_csv(path_or_buf = "/Users/hoanjinan_otoko/Desktop/epam-ds-training/greenfox/hoanjinan/week-07/normalised_data.csv")

